## Домашнее задание №4. GAN

### Creating config object (argparse workaround)

In [ ]:
student_name = "Fedor Petriaikin" # Введите свое имя и фамилию

В рамках этого домашнего задания мы разберем архитектуру простейшего GAN для генерации картинок одежды и попробуем ее немного модифицировать

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset

import torchvision
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
# Конфигурация
num_epochs = 3
noise_size = 50
print_freq = 100
batch_size = 16

# Как мы будем помечать истинные и сгенерированные изображения
real_label = 1
fake_label = 0

# Шум, из которого генератор будет делать картинки
# Заготовка - новые можно сделать с помощью .normal_()
noise = torch.FloatTensor(batch_size, noise_size)

In [ ]:
# Отображает 16 картинок из pic в сетке 4x4
def show_pictures(pic):
    plt.figure(figsize=(6, 7))
    for i in range(16):
        plt.subplot(4, 4, i + 1)
        plt.imshow(pic[i].detach().numpy().reshape(28, 28), cmap=plt.cm.Greys_r)
        plt.axis('off')

**Загружаем данные для работы**

In [ ]:
train = torchvision.datasets.FashionMNIST("fashion_mnist", train=True,
                                          transform=torchvision.transforms.ToTensor(), download=True)

In [ ]:
dataloader = DataLoader(train, batch_size, shuffle=True)
for batch, _ in dataloader: # Жертвуем 1 батчом из первой итерации для обучения - ничего страшного
    break

In [ ]:
show_pictures(batch.reshape(-1, 28, 28))

**Реализация GAN**

In [ ]:
class Generator(nn.Module):
    def __init__(self):
        super(Generator, self).__init__()
        self.model = nn.Sequential( 
            nn.Linear(noise_size, 200),
            nn.ReLU(inplace=True),
            nn.Linear(200, 28*28),
            nn.Sigmoid())
        
    def forward(self, x):
        return self.model(x)
    
class Discriminator(nn.Module):
    def __init__(self):
        super(Discriminator, self).__init__()
        self.model = nn.Sequential(
            nn.Linear(28*28, 200),
            nn.ReLU(inplace=True),
            nn.Linear(200, 50),
            nn.ReLU(inplace=True),
            nn.Linear(50, 1), 
            nn.Sigmoid())
    def forward(self, x):
        return self.model(x)

In [ ]:
generator = Generator()
discriminator = Discriminator()

In [ ]:
optim_G = optim.Adam(params=generator.parameters(), lr=0.0001)
optim_D = optim.Adam(params=discriminator.parameters(), lr=0.0001)

criterion = nn.BCELoss()

In [ ]:
ERRD_x = np.zeros(num_epochs)
ERRD_z = np.zeros(num_epochs)
ERRG = np.zeros(num_epochs)
N = len(dataloader)

for epoch in range(num_epochs):
    for iteration, (images, cat) in enumerate(dataloader):
        ####### 
        # Discriminator stage: maximize log(D(x)) + log(1 - D(G(z))) 
        #######
        discriminator.zero_grad()
        label = torch.FloatTensor(batch_size) # место для меток
        
        # real
        label.data.fill_(real_label)
        input_data = images.view(images.shape[0], -1)
        output = discriminator(input_data)
        errD_x = criterion(output, label)
        ERRD_x[epoch] += errD_x.item()
        errD_x.backward()
        
        # fake 
        noise.data.normal_(0, 1)
        fake = generator(noise)
        label.data.fill_(fake_label)
        output = discriminator(fake.detach())
        errD_z = criterion(output, label)
        ERRD_z[epoch] += errD_z.item()
        errD_z.backward()
        
        optim_D.step()
        
        ####### 
        # Generator stage: maximize log(D(G(x))
        #######
        generator.zero_grad()
        label.data.fill_(real_label)
        output = discriminator(fake)
        errG = criterion(output, label)
        ERRG[epoch] += errG.item()
        errG.backward()
        
        optim_G.step()
        
        if (iteration+1) % print_freq == 0:
            print('Epoch:{} Iter: {} errD_x: {:.2f} errD_z: {:.2f} errG: {:.2f}'.format(epoch+1,
                                                                                            iteration+1, 
                                                                                            errD_x.item(),
                                                                                            errD_z.item(), 
                                                                                            errG.item()))

In [ ]:
# Смотрим на результат
noise.data.normal_(0, 1)
fake = generator(noise)
print(fake.shape)

show_pictures(fake)

### Задание

1) Посмотрите на реализацию GAN выше. Постройте интерполяцию между какими-нибудь двумя сгенерированными картинками.  
  (Опционально) Добавьте свертки в генератор и дискриминатор, как в статье про DCGAN.

2) Поменяйте реализацию, чтобы получился LSGAN https://arxiv.org/pdf/1611.04076v2.pdf

3) Добавьте к обучению GAN условие на метку, продемонстрируйте условную генерацию. https://arxiv.org/pdf/1411.1784.pdf

4) Напишите отчет что попробовали, какие результаты получили, как вам кажется надо обучать GAN, чтобы добиться сходимости?

В каждом пункте постройте графики функций потерь.  
Спасибо за выполнение заданий!